In [ ]:
import sys
from pathlib import Path
# set the notebook's CWD to your repo root
%cd D:/deepdemand
ROOT = Path.cwd().parents[0]   # go up one level
sys.path.insert(0, str(ROOT))

In [5]:
# ===========================================
# Signed error map with 8 layers (Train/Test × GEH/MAE × Over/Under)
# - Popups show edge_id, GT, Pred, GEH (+signed), MAE (+signed)
# - Color = red (over), blue (under); thickness/opacity scale with magnitude
# ===========================================

import json
from pathlib import Path
import numpy as np
import geopandas as gpd
import folium

# --- paths ---
GEOJSON_PATH = "data/highway_network/uk_driving_edges_simplified.geojson"
TRAIN_JSON   = "interpret/pred_results_train.json"
TEST_JSON    = "interpret/pred_results_test.json"
OUT_HTML     = "interpret/errors_map_8layers.html"

# ===========================================
# Load sensor mapping
# ===========================================
with open("data/traffic_volume/edge_to_sensor.json", "r") as f:
    edge_to_sensor = json.load(f)

def sensors_for_edge(eid):
    arr = edge_to_sensor.get(eid, [])
    if not arr:
        return "None"
    return ", ".join(str(x) for x in arr)

# --- load preds ---
def load_preds(json_path):
    with open(json_path, "r") as f:
        rows = json.load(f)
    # {edge_id: (gt, pred)}
    return { str(r["edge_id"]): (float(r["gt"]), float(r["pred"])) for r in rows }

preds_train = load_preds(TRAIN_JSON)
preds_test  = load_preds(TEST_JSON)

# --- read edges ---
gdf = gpd.read_file(GEOJSON_PATH)
try:
    if gdf.crs is None:
        gdf.set_crs(epsg=4326, inplace=True)
    elif gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(epsg=4326)
except Exception:
    pass

# --- helpers ---
def edge_id_from_props(u, v, key):
    for val_name in ("u","v","key"):
        pass
    try:
        u = int(u)
    except Exception:
        pass
    try:
        v = int(v)
    except Exception:
        pass
    try:
        key = int(key)
    except Exception:
        pass
    return f"{u}_{v}_{key}"

def error_record(gt, pred, eps=1e-9):
    err = pred - gt
    mae = abs(err)
    geh = np.sqrt(2.0 * (err**2) / (max(pred + gt, eps)))
    return {
        "gt": float(gt),
        "pred": float(pred),
        "signed_mae": float(err),
        "mae": float(mae),
        "signed_geh": float(np.sign(err) * geh),
        "geh": float(geh),
    }

def build_error_dict(preds_dict):
    out = {}
    for eid, (gt, pred) in preds_dict.items():
        out[eid] = error_record(gt, pred)
    return out

err_train = build_error_dict(preds_train)
err_test  = build_error_dict(preds_test)

# global maxima for styling (consistent scales across layers)
absmax_geh = max(
    [*( (rec["geh"] for rec in err_train.values()) ), *( (rec["geh"] for rec in err_test.values()) )],
    default=0.0
)
absmax_mae = max(
    [*( (rec["mae"] for rec in err_train.values()) ),  *( (rec["mae"] for rec in err_test.values()) )],
    default=0.0
)

# ===========================================
# NEW styling rules (fixed opacity, percentile-based width)
# ===========================================

FIXED_OPACITY = 1.0
MIN_WEIGHT    = 0.8
MAX_WEIGHT    = 10.0

def percentile_cutoff(values, p=90):
    if len(values) == 0:
        return 0.0
    return float(np.percentile(values, p))


# compute percentiles for GEH and MAE
geh_all = [rec["geh"] for rec in err_train.values()] + \
          [rec["geh"] for rec in err_test.values()]

mae_all = [rec["mae"] for rec in err_train.values()] + \
          [rec["mae"] for rec in err_test.values()]

geh_p90 = percentile_cutoff(geh_all, 95)
mae_p90 = percentile_cutoff(mae_all, 95)

def color_for_signed(signed_val):
    return "#d73027" if signed_val > 0 else "#4575b4"  # red over / blue under

def thickness_from_magnitude(mag, p90):
    """
    If mag >= p90 → MAX_WEIGHT.
    Otherwise linearly interpolate between MIN_WEIGHT → MAX_WEIGHT.
    """
    if mag >= p90:
        return MAX_WEIGHT
    # linear scale 0 → p90 maps to MIN_WEIGHT → MAX_WEIGHT
    ratio = mag / max(p90, 1e-9)
    return MIN_WEIGHT + ratio * (MAX_WEIGHT - MIN_WEIGHT)

# ===========================================
# UPDATED add_layer_lines() to incorporate sensors + new thickness rule
# ===========================================
def add_layer_lines(feature_group, err_dict, which_metric: str, sign_filter: str, label_prefix: str):
    """
    which_metric: 'geh' or 'mae'
    sign_filter : 'over' or 'under'
    """
    p90 = geh_p90 if which_metric == "geh" else mae_p90

    for _, row in gdf.iterrows():
        eid = edge_id_from_props(row.get("u"), row.get("v"), row.get("key"))
        rec = err_dict.get(eid)
        if rec is None:
            continue

        if which_metric == "geh":
            signed_val = rec["signed_geh"]
            mag = rec["geh"]
        else:
            signed_val = rec["signed_mae"]
            mag = rec["mae"]

        # filter sign
        if sign_filter == "over" and signed_val <= 0:
            continue
        if sign_filter == "under" and signed_val >= 0:
            continue

        color = color_for_signed(signed_val)
        weight = thickness_from_magnitude(mag, p90)
        opacity = FIXED_OPACITY

        sensor_str = sensors_for_edge(eid)

        # popup HTML
        popup_html = (
            f"<b>{label_prefix}</b><br>"
            f"<b>edge_id:</b> {eid}<br>"
            f"<b>Sensors:</b> {sensor_str}<br>"
            f"<b>GT:</b> {rec['gt']:,.0f} &nbsp; <b>Pred:</b> {rec['pred']:,.0f}<br>"
            f"<b>GEH:</b> {rec['geh']:.3f} ({rec['signed_geh']:+.3f})<br>"
            f"<b>MAE:</b> {rec['mae']:.0f} ({rec['signed_mae']:+.0f})"
        )

        tooltip = (
            f"{label_prefix} | {eid} | Sensors={sensor_str} | "
            f"GT={rec['gt']:,.0f}, Pred={rec['pred']:,.0f} | "
            f"GEH={rec['geh']:.3f} ({rec['signed_geh']:+.3f}), "
            f"MAE={rec['mae']:.0f} ({rec['signed_mae']:+.0f})"
        )

        geom = row.geometry
        if geom is None or geom.is_empty:
            continue

        def _add_line(coords):
            folium.PolyLine(
                coords, color=color, weight=weight, opacity=opacity,
                popup=folium.Popup(popup_html, max_width=360),
                tooltip=tooltip
            ).add_to(feature_group)

        if geom.geom_type == "LineString":
            coords = [(lonlat[1], lonlat[0]) for lonlat in geom.coords]
            _add_line(coords)

        elif geom.geom_type == "MultiLineString":
            for ls in geom.geoms:
                coords = [(lonlat[1], lonlat[0]) for lonlat in ls.coords]
                _add_line(coords)

# --- build map ---
bounds = gdf.total_bounds
m = folium.Map(location=[(bounds[1]+bounds[3])/2, (bounds[0]+bounds[2])/2], zoom_start=6, tiles="cartodbpositron")

# 8 layers: Train/Test × (GEH, MAE) × (Over, Under)
layers = {
    # GEH-driven styling
    "Train — GEH Over":  (err_train, "geh", "over",  True),
    "Train — GEH Under": (err_train, "geh", "under", True),
    "Test — GEH Over":   (err_test,  "geh", "over",  True),
    "Test — GEH Under":  (err_test,  "geh", "under", True),
    # MAE-driven styling
    "Train — MAE Over":  (err_train, "mae", "over",  False),
    "Train — MAE Under": (err_train, "mae", "under", False),
    "Test — MAE Over":   (err_test,  "mae", "over",  False),
    "Test — MAE Under":  (err_test,  "mae", "under", False),
}

for name, (err_dict, which_metric, sign_filter, _) in layers.items():
    fg = folium.FeatureGroup(name=name, show=("GEH" in name))
    add_layer_lines(fg, err_dict, which_metric, sign_filter, name)
    fg.add_to(m)

# legend
legend_html = """
<div style="
     position: fixed; bottom: 20px; left: 20px; z-index: 9999;
     background: white; padding: 10px 12px; border: 1px solid #ccc; border-radius: 6px;
     font-size: 12px; line-height: 1.4;">
  <b>Signed error legend</b><br>
  <span style="color:#d73027;">&#9608;</span> Overestimation (pred &gt; gt) — red<br>
  <span style="color:#4575b4;">&#9608;</span> Underestimation (pred &lt; gt) — blue<br>
  Thickness/opacity &propto; |error| (GEH or MAE)
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

folium.LayerControl(collapsed=False).add_to(m)
m.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])

Path(OUT_HTML).parent.mkdir(parents=True, exist_ok=True)
m.save(OUT_HTML)
print(f"Saved map to: {OUT_HTML}")

Saved map to: interpret/errors_map_8layers.html
